# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `Planet` module.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet-python-client](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)

You should also have an account on the Planet Platform. You will login to create an access token that authenticates your API key from the [account page](https://www.planet.com/account/).

## Useful links 
* [Planet SDK and Developer Resources](https://docs.planet.com/develop/sdks/#planet-sdk-for-python-and-cli/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Data API Documentation](https://docs.planet.com/develop/apis/data/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API.

The basic workflow for interacting with the Data API is:
1. Search item types based on filters
2. Activate assets
3. Download assets

Additional features like saving searches and getting summary statistics are included at the end of the tutorial.

____

## Set up

In order to interact with the Planet API using the client, we need to import the necessary packages & define helper functions.

In [ ]:
from datetime import datetime
from planet import Auth, Planet
from planet import Session, data_filter
import json

# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

## SDK Authentication

We next need to create an OAuth2 access token using our Planet login for python client access. 
If you are not currently logged in to your Planet account, you will be prompted via a login link to do so. 
For more information, please visit https://docs.planet.com/develop/authentication

In [ ]:
# OAuth2 python client authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = Auth.from_profile('planet-user', save_state_to_storage=True)
if not auth.is_initialized():
    auth.user_login(allow_open_browser=False, allow_tty_prompt=True)

session = Session(auth)
pl = Planet(session)

Let's also read in a GeoJSON geometry into a variable so we can use it during testing.

In [ ]:
with open("data/san-francisco.json") as f:
    geom = json.loads(f.read())

In [ ]:
geom

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

### Filters

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API.
- Here we are searching for "REOrthoTile" and "PSScene" item types with `and_filter` that are:
    - 90 `clear_percent` or more from cloud, haze, shadow, or snow.
    - `acquired` after January 1st, 2017
    - Have `cloud_cover` value of less than 0.1


In [ ]:
# Define the filters we'll use to find our data

item_types = ["REOrthoTile", "PSScene"]

geom_filter = data_filter.geometry_filter(geom)
clear_percent_filter = data_filter.range_filter('clear_percent', 90)
date_range_filter = data_filter.date_range_filter("acquired", gt = datetime(month=1, day=1, year=2017))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.1)

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter, cloud_cover_filter])

In [ ]:
combined_filter

Now let's build the request:

In [ ]:
async with Session() as sess:
    cl = sess.client('data')
    request = await cl.create_search(name='planet_client_demo',search_filter=combined_filter, item_types=item_types)

In [ ]:
request

In [ ]:
# Search the Data API

# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100.
# Here, we're setting our result limit to 50.

async with Session() as sess:
    cl = sess.client('data')
    items = cl.run_search(search_id=request['id'], limit=50)
    item_list = [i async for i in items]

Now, we can iterate through our search results.

In [ ]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

## Assets and downloads

After a search returns results, the Python client can be used to check for assets and initiate downloads. Let's start by looking at one item and the assets available to download for that item.

For more information on Items and Assets, check out [Items & Assets](https://developers.planet.com/docs/apis/data/items-assets/) on the Planet Developer Center.

In [ ]:
# As an example, let's look at the first result in our item_list and grab the item_id and item_type:
item = item_list[0]

item_id = item['id']
item_type = item['properties']['item_type']


print(item_id, item_type)

In [ ]:
# The list of assets for an item that a user has access to can be retrieved with `permissions`
item['_permissions']

There are a few steps involved in order to download an asset using the Planet Python Client:

* **Get Asset:** Get a description of our asset based on the specifications we're looking for
* **Activate Asset:** Activate the asset with the given description
* **Wait Asset:** Wait for the asset to be activated
* **Download Asset:** Now our asset is ready for download!

Let's go through these steps below. We'll do this for our analytic asset, as well as the analytic_xml asset.

In [ ]:
# Analytic Asset
async with Session() as sess:
    cl = sess.client('data')
    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='ortho_analytic_4b')

    # Activate Asset
    await cl.activate_asset(asset=asset_desc)

    # Wait Asset
    await cl.wait_asset(asset=asset_desc)

    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory='output', overwrite=True)

In [ ]:
# Analytic XML Asset
async with Session() as sess:
    cl = sess.client('data')

    # Get Asset
    asset_desc = await cl.get_asset(item_type_id=item_type,item_id=item_id, asset_type_id='ortho_analytic_4b_xml')

    # Activate Asset
    await cl.activate_asset(asset=asset_desc)

    # Wait Asset (this may take some time!)
    await cl.wait_asset(asset=asset_desc)

    # Download Asset
    asset_path = await cl.download_asset(asset=asset_desc, directory='output', overwrite=True)

Congratulations! Both the `ortho_analytic_4b` and `ortho_analytic_4b_xml` assets should be saved in our `output` directory.

_______

# Other Data-API Features

## Saved Searches

The Data API client can also help in managing saved searches on the Planet Platform.

In [ ]:
async with Session() as sess:
    cl = sess.client('data')
    searches = cl.list_searches()
    searches_list = [i async for i in searches]

In [ ]:
# We may have a lot of saved searches!

len(searches_list)

View your saved searches. Here we're viewing our first 50 results.

In [ ]:
for search in searches_list[:50]:
    print(search['id'], search['name'])

We can find the saved search we're looking for by filtering on our search id:

In [ ]:
async with Session() as sess:
    cl = sess.client('data')
    search = await cl.get_search(searches_list[0]['id'])

In [ ]:
search

# Statistics

The Data API client can also help report statistical summaries of the amount of data in the Planet API.

In [ ]:
# Here, we can look at the stats for the search we've been working with:

async with Session() as sess:
    cl = sess.client('data')
    stats = await cl.get_stats(interval='year', search_filter=combined_filter, item_types=item_types)

In [ ]:
indent(stats)